In [ ]:
import json
import os
from copy import deepcopy
from typing import List
from uuid import uuid4 as uuid

import keras
import numpy as np
from matplotlib import pyplot as plt

from src.models.geometric_figure import GeometricFigure
from src.services.geometric_figure import (create_damaged_geometric_figures,
                                           damage_image, get_geometric_figures,
                                           get_input_and_output,
                                           plot_geometric_figures,
                                           prediction_to_category,
                                           prediction_to_is_failed)
from src.utils.files import create_directory_if_not_exists

In [ ]:
IMAGE_SIZE = (128, 128)
DATA_VERSION = '2023-04-12'
NEW_DATA_VERSION = '2023-04-24'
NUMBER_OF_REPETITIONS_TO_CREATE_DATA = 1

In [ ]:
geometric_figures: List[GeometricFigure] = get_geometric_figures(f'data/{DATA_VERSION}', IMAGE_SIZE)
print(f'Loaded {len(geometric_figures)} geometric figures')

In [ ]:
def create_geometric_figures_variations(geometric_figures: List[GeometricFigure]) -> List[GeometricFigure]:
    xs, ys = zip(*[get_input_and_output(gf) for gf in geometric_figures])
    xs, ys = np.array(xs), np.array(ys)
    data_generator = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
    )
    train_generator = data_generator.flow(xs, ys, batch_size=1)
    created_geometric_figures = []
    for _ in range(len(geometric_figures)):
        x, argmax = train_generator.next()
        x, argmax = x[0], argmax[0]
        y = np.zeros(3)
        y[argmax] = 1
        geometric_figure: GeometricFigure = {
            'id': 0,
            'category': prediction_to_category(y),
            'is_failed': prediction_to_is_failed(y),
            'filename': f'{uuid()}.jpg',
            'image': x,
        }
        created_geometric_figures.append(geometric_figure)
    return created_geometric_figures

In [ ]:
geometric_figures_variations = []
for _ in range(NUMBER_OF_REPETITIONS_TO_CREATE_DATA):
    new_geometric_figures_variations = create_geometric_figures_variations(geometric_figures)
    geometric_figures_variations.extend(new_geometric_figures_variations)
print(f'Created {len(geometric_figures_variations)} geometric figures variations')

In [ ]:
damaged_geometric_figures = []
for _ in range(NUMBER_OF_REPETITIONS_TO_CREATE_DATA):
    new_damaged_geometric_figures = create_damaged_geometric_figures(geometric_figures)
    damaged_geometric_figures.extend(new_damaged_geometric_figures)
print(f'Created {len(damaged_geometric_figures)} damaged geometric figures')

In [ ]:
number_of_plots = 7*1
columns = 7
random_geometric_figures = [geometric_figures[i] for i in np.random.randint(0, len(geometric_figures), number_of_plots)]
random_damaged_geometric_figures = deepcopy(random_geometric_figures)
for gf in random_damaged_geometric_figures:
    gf['image'] = damage_image(gf['image'])
plot_geometric_figures(random_geometric_figures, columns, cmap='gray')
plot_geometric_figures(random_damaged_geometric_figures, columns, cmap='gray')
plt.show()

In [ ]:
path = os.path.join('data', NEW_DATA_VERSION)
create_directory_if_not_exists(os.path.join(path, 'Geometric Figures', ''))


def solve_damaged_geometric_figure(damaged_geometric_figure: GeometricFigure) -> GeometricFigure:
    gf = deepcopy(damaged_geometric_figure)
    gf['image'] = np.repeat(gf['image'], 3, axis=2)
    gf['filename'] = str(uuid()) + '.jpg'
    return gf

def solve_geometric_figure_variation(geometric_figure_variation: GeometricFigure) -> GeometricFigure:
    gf = deepcopy(geometric_figure_variation)
    gf['image'] = np.repeat(gf['image'], 3, axis=2)
    gf['filename'] = str(uuid()) + '.jpg'
    return gf


new_geometric_figures = []
new_geometric_figures.extend([solve_geometric_figure_variation(gf) for gf in geometric_figures_variations])
new_geometric_figures.extend([solve_damaged_geometric_figure(gf) for gf in damaged_geometric_figures])

for gf in new_geometric_figures:
    plt.imsave(os.path.join(path, 'Geometric Figures', gf['filename']), gf['image'])

In [ ]:
def convert_geometric_figure_to_data(geometric_figure: GeometricFigure) -> dict:
    return {
        'filename': geometric_figure['filename'],
        'id': geometric_figure['id'],
        'isFailed': geometric_figure['is_failed'],
        'type': geometric_figure['category']
    }


data = [convert_geometric_figure_to_data(gf) for gf in new_geometric_figures]
with open(os.path.join(path, 'data.json'), 'w') as f:
    json.dump(data, f, indent=2)